In [1]:
pip list

Package                   Version
------------------------- -----------
annotated-types           0.7.0
anthropic                 0.49.0
anyio                     4.9.0
asttokens                 3.0.0
attrs                     25.3.0
beautifulsoup4            4.13.3
bleach                    6.2.0
certifi                   2025.1.31
charset-normalizer        3.4.1
click                     8.1.8
comm                      0.2.2
contourpy                 1.3.1
cycler                    0.12.1
debugpy                   1.8.11
decorator                 5.2.1
defusedxml                0.7.1
distro                    1.9.0
exceptiongroup            1.2.2
executing                 2.1.0
fastapi                   0.115.12
fastjsonschema            2.21.1
filelock                  3.18.0
fonttools                 4.56.0
fsspec                    2025.3.2
greenlet                  3.1.1
groq                      0.22.0
h11                       0.14.0
httpcore                  1.0.7
httpx       

In [3]:
import uvicorn
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from typing import List
from fastapi import FastAPI

app = FastAPI()

# Define Models
class Fruit(BaseModel):
    name: str

class Fruits(BaseModel):
    fruits: List[Fruit]

# CORS Middleware
origins = [
    "http://localhost:3000",  # Corrected URL format
]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,  # This must be a list
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# In-memory database
memory_db = {"fruits": []}

# Get Fruits
@app.get("/fruits", response_model=Fruits)
def get_fruits():
    return Fruits(fruits=memory_db["fruits"])

# Add a Fruit
@app.post("/fruits", response_model=Fruit)
def add_fruit(fruit: Fruit):
    memory_db["fruits"].append(fruit)
    return fruit

# Run the app
if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)


RuntimeError: asyncio.run() cannot be called from a running event loop

In [ ]:
from fastapi import FastAPI,Depends,HTTPException,Header
from pydantic import BaseModel
import os
from dotenv import load_dotenv

load_dotenv()
import ollama
API_KEY_CREDITS={os.getenv("API_KEY"):10}
app = FastAPI()

class PromptRequest(BaseModel):
    prompt: str


def verify_api_key(x_api_key:str=Header(None)):
    credits=API_KEY_CREDITS.get(x_api_key,0)
    if credits<=0:
        raise HTTPException(status_code=401,detail="no")
    return x_api_key    
    




@app.post("/generate")
def generate(request: PromptRequest,x_api_key=Depends(verify_api_key)):
    API_KEY_CREDITS[x_api_key]-=1
    try:
        response = ollama.chat(
            model="gemma3:1b",
            messages=[{"role": "user", "content": request.prompt}]
        )
        return {"response": response["message"]["content"]}
    except Exception as e:
        return {"error": str(e)}
    


